In [2]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment,Dataset,Datastore

In [3]:
#used my own azure pay as go account.

workspace=Workspace.create(name='azuremlsdk-ws01',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',
resource_group='azuremlsdk-rg01',create_resource_group=True,
location='eastus2')

#workspace=Workspace.from_config('config.json')

Deploying StorageAccount with name azuremlsstorage28881ed14.
Deploying KeyVault with name azuremlskeyvaultd52adada.
Deploying AppInsights with name azuremlsinsightscb9df6a2.
Deployed AppInsights with name azuremlsinsightscb9df6a2. Took 1.97 seconds.
Deployed KeyVault with name azuremlskeyvaultd52adada. Took 18.66 seconds.
Deployed StorageAccount with name azuremlsstorage28881ed14. Took 23.71 seconds.
Deploying Workspace with name azuremlsdk-ws01.
Deployed Workspace with name azuremlsdk-ws01. Took 23.06 seconds.


In [14]:
ds=Datastore.register_azure_blob_container(workspace=workspace,account_name='storagerrg01',datastore_name='diabetesdatastore',account_key='VArbyTAf+xp2LiT/RKKSNU9/hRIjNQmBXYPWefi0/YpaABUYbRcPCNcCX/cjZ21oOJHvBtjoPQPo+AStwG+Odg==',container_name='diabetes',create_if_not_exists=True)



In [15]:
ds

{
  "name": "diabetesdatastore",
  "container_name": "diabetes",
  "account_name": "storagerrg01",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [16]:
path=[(ds,'diabetes.csv')]
dataset=Dataset.Tabular.from_delimited_files(path=path)

In [17]:
dataset=dataset.register(workspace=workspace,name='diabetesdataset')

In [20]:
dataset1=dataset.to_pandas_dataframe()

In [68]:
ds_batch=Datastore.register_azure_blob_container(workspace=workspace,account_name='storagerrg01',datastore_name='diabetesdatastorebatchtest',account_key='VArbyTAf+xp2LiT/RKKSNU9/hRIjNQmBXYPWefi0/YpaABUYbRcPCNcCX/cjZ21oOJHvBtjoPQPo+AStwG+Odg==',container_name='batch',create_if_not_exists=True)


In [89]:
dataset_batch=Dataset.File.from_files(path=[(ds_batch,'1.csv'),(ds_batch,'2.csv'),(ds_batch,'3.csv'),(ds_batch,'4.csv'),(ds_batch,'5.csv'),(ds_batch,'6.csv'),(ds_batch,'7.csv'),(ds_batch,'8.csv'),(ds_batch,'9.csv'),(ds_batch,'10.csv')])
dataset_batch=dataset_batch.register(workspace=workspace,name='diabetesbatchtest1',create_new_version=True)

In [90]:
dataset_batch

{
  "source": [
    "('diabetesdatastorebatchtest', '1.csv')",
    "('diabetesdatastorebatchtest', '2.csv')",
    "('diabetesdatastorebatchtest', '3.csv')",
    "('diabetesdatastorebatchtest', '4.csv')",
    "('diabetesdatastorebatchtest', '5.csv')",
    "('diabetesdatastorebatchtest', '6.csv')",
    "('diabetesdatastorebatchtest', '7.csv')",
    "('diabetesdatastorebatchtest', '8.csv')",
    "('diabetesdatastorebatchtest', '9.csv')",
    "('diabetesdatastorebatchtest', '10.csv')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "91c3e8db-5e55-4d91-8643-d12913fa6278",
    "name": "diabetesbatchtest1",
    "version": 2,
    "workspace": "Workspace.create(name='azuremlsdk-ws01', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='azuremlsdk-rg01')"
  }
}

In [96]:
dataset1

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [21]:
env=Environment(workspace=workspace,name='diabetesenv')
Exp=Experiment(workspace=workspace,name='diabetesexp')
run=Exp.start_logging(snapshot_directory=None)
from sklearn.model_selection import train_test_split
x=dataset1.iloc[:,:-1].values
y=dataset1.iloc[:,-1].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
fitted_model=RFC.fit(x_train,y_train)
y_pred=fitted_model.predict(x_test)
from sklearn.metrics import accuracy_score
run.log('accuracy',accuracy_score(y_test,y_pred))
import joblib
path='outputs/model_rfc.pkl'
joblib.dump(value=fitted_model,filename=path)
run.complete()

In [80]:
dataset_batch

{
  "source": [
    "('diabetesdatastorebatchtest', '1.csv')",
    "('diabetesdatastorebatchtest', '3.csv')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "9cc3d041-c263-4c94-8bb2-7f4ba925d430",
    "name": "diabetesbatchtest",
    "version": 1,
    "workspace": "Workspace.create(name='azuremlsdk-ws01', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='azuremlsdk-rg01')"
  }
}

In [34]:
#registering the model
run_id=workspace.get_run('be447e27-cc2a-4b70-941e-224bbb912cc9')
model=run_id.register_model(model_name='diabetesmodel',model_path='outputs/model_rfc.pkl')

In [37]:
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import ParallelRunStep,ParallelRunConfig

In [100]:
import os
experiment_folder='batch_pipeline'
os.makedirs(experiment_folder,exist_ok=True)

In [38]:
output_intermediary=PipelineData(name='intermediary',datastore=ds)

In [107]:
%%writefile $experiment_folder/batch_diabetes.py

import pandas as pd
import numpy as np 
from azureml.core.model import Model

def init():
    global model
    model_path=Model.get_model_path(model_name='diabetesmodel')
    model=joblib.load(model_path)

def run(mini_batch):
    results=[]
    for f in mini_batch:
        data=np.genfromtxt(f,delimiter=',')
        y_pred=model.predict(data.reshape(-1,1))
        results.append('{}:{}'.format(os.path.basename(f),y_pred[0])
    return results

Overwriting batch_pipeline/batch_diabetes.py


In [103]:
env=Environment(workspace=workspace,name='diabetesenv')
from azureml.core import conda_dependencies
dep=conda_dependencies.CondaDependencies.create(pip_packages=['scikit-learn','pandas','azureml-defaults','azureml-core'])
env.python.conda_dependencies=dep
env.register(workspace=workspace)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220616.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "diabetesenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [93]:
from azureml.core.compute import AmlCompute,ComputeTarget
compute_name='cpucluster'
cluster_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,vm_priority='lowpriority')
compute_target=ComputeTarget.create(workspace=workspace,name=compute_name,provisioning_configuration=cluster_config)
compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [79]:
compute_target.delete()

In [109]:

parralel_config=ParallelRunConfig(source_directory=experiment_folder,
entry_script='batch_pipeline.py',
mini_batch_size='1',
error_threshold=10,
output_action='append_row',
environment=env,
node_count=1,
compute_target=compute_target)

parralel_step=ParallelRunStep(name='batch_diabetes',parallel_run_config=parralel_config,
inputs=[dataset_batch.as_named_input('diabetesbatchtest')],
output=output_intermediary,allow_reuse=True)


In [110]:
from azureml.pipeline.core import Pipeline
pipeline=Pipeline(workspace=workspace,steps=[parralel_step])
pipeline_run=pipeline.submit(experiment_name='batch_pipeline')
pipeline_run.wait_for_completion(show_output=True)

Created step batch_diabetes [c2ed9d4b][d88dc73f-d933-456f-b986-181e5533ac66], (This step will run and generate new outputs)
Submitted PipelineRun 632b0a04-04f6-440f-b26d-a297298f17b6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/632b0a04-04f6-440f-b26d-a297298f17b6?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRunId: 632b0a04-04f6-440f-b26d-a297298f17b6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/632b0a04-04f6-440f-b26d-a297298f17b6?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530
PipelineRun Status: Running


StepRunId: 87f04cca-2ca1-49b4-8c75-04b03cede7ab
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/87f04cca-2ca1-49b4-8c75-04b03cede7ab?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegrou

'Finished'